In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import seaborn as sns
from scipy import stats
from scipy.stats import norm

# VERİ ÇERÇEVESİNİ TANIMAK

In [ ]:
file_path="../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv"
df = pd.read_csv(file_path)
df.head()


In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

Buraya kadar en çok dikkatimizi çeken şey, TotalCharges kolonunun df.head() içerisinde nümerik değer aldığını görmemize rağmen
veri tipinin object olarak görülmesi. Bununla ileride, Keşifsel Veri Analizi kısmında ilgileneceğiz. Bunun dışında eksik değerlerimizin 
olmadığı, genelde object değerlerden oluşan bir veri seti üzerinde çalışıyoruz.

In [ ]:
df.describe()

Burada da gözüktüğü üzere TotalCharges sütunu alınmamış.

In [ ]:
df.nunique()

Burada object türlü verilerin birkaçının dağılımın yüzdesine bakalım:

In [ ]:
plt.figure(figsize=(18,4))

plt.subplot(1,3,1)
plt.pie(df.groupby("MultipleLines")["MultipleLines"].count(), labels=df.groupby("MultipleLines")["MultipleLines"].count().index, autopct='%1.1f%%',
        shadow=True, startangle=90)
plt.title('MultipleLines')
plt.subplot(1,3,2)
plt.pie(df.groupby("OnlineSecurity")["OnlineSecurity"].count(), labels=df.groupby("OnlineSecurity")["OnlineSecurity"].count().index, autopct='%1.1f%%',
        shadow=True, startangle=90)
plt.title('OnlineSecurity')
plt.subplot(1,3,3)
plt.pie(df.groupby("TechSupport")["TechSupport"].count(), labels=df.groupby("TechSupport")["TechSupport"].count().index, autopct='%1.1f%%',
        shadow=True, startangle=90)
plt.title('TechSupport')
plt.show()

# İstatistiğe Giriş

Bazı temel istatistik kavramları ile veri setimizdeki float türündeki değerleri inceleyelim.

***Ortalama***

In [ ]:
print(np.mean(df['tenure']))
print(np.mean(df['MonthlyCharges']))

***Medyan***

In [ ]:
print(np.median(df['tenure']))
print(np.median(df['MonthlyCharges']))

***Mod***

In [ ]:
statistics.mode(df['tenure'])
statistics.mode(df['MonthlyCharges'])

***Varyans***

In [ ]:
print(df['tenure'].var())
print(df['MonthlyCharges'].var())

***Standart Hata***

In [ ]:
print(stats.sem(df['tenure']))
print(stats.sem(df['MonthlyCharges']))

**Dağılımlar**

***Normal Dağılım***

In [ ]:
mu, sigma, n = 0.5, 0.1, 10000
plt.figure(figsize=(10,12), dpi = 100)
plt.subplot(2,1,1)
count, bins, ignore = plt.hist(df["tenure"], 50, density=True)
plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi))*np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
         linewidth=3, color='y')
plt.title("tenure")
plt.subplot(2,1,2)
count, bins, ignore = plt.hist(df["MonthlyCharges"], 50, density=True)
plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi))*np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
         linewidth=3, color='y')
plt.title("MonthlyCharges")
plt.show()

İki değişkenimiz de normal dağılım olmaktan çok uzak.

In [ ]:
mu, sigma, n = 0.5, 0.1, 10000
plt.figure(figsize=(10,12), dpi = 100)
plt.subplot(2,1,1)
plt.hist(df["tenure"], bins=20)
plt.axvline(np.mean(df["tenure"]), linestyle='solid', linewidth=2)
plt.axvline(np.mean(df["tenure"]) + np.std(df["tenure"]), 
            linestyle='dashed', linewidth=2)
plt.axvline(np.mean(df["tenure"]) - np.std(df["tenure"]), 
            linestyle='dashed', linewidth=2)
plt.title("tenure")
plt.subplot(2,1,2)
plt.hist(df["MonthlyCharges"], bins=20)
plt.axvline(np.mean(df["MonthlyCharges"]), linestyle='solid', linewidth=2)
plt.axvline(np.mean(df["MonthlyCharges"]) + np.std(df["MonthlyCharges"]), 
            linestyle='dashed', linewidth=2)
plt.axvline(np.mean(df["MonthlyCharges"]) - np.std(df["MonthlyCharges"]), 
            linestyle='dashed', linewidth=2)
plt.title("MonthlyCharges")
plt.show()

***Popülasyon***

In [ ]:
print('Tenure oranı: ', len(df[df["tenure"] > 30])/len(df["tenure"])*100) #Tenure 'ın 30'dan büyük olma oranı
print('MonthlyCharges oranı: ', len(df[df["MonthlyCharges"] > 75])/len(df["MonthlyCharges"])*100) #MonthlyCharges'ın 75'ten büyük olma oranı

# Keşifsel Veri Analizi

***Veri Temizleme ve Eksik Veriler***

TotalCharges sütununun float değer göstermesine rağmen veri tipinin object olduğunu görmüştük.
 Şimdi gerçek tipine çevirebiliriz.

In [ ]:
for sutun_adi in ["TotalCharges"]:
    print("{} sütunu için problemli değerler : ".format(sutun_adi))
    hatali_degerler = []
    for deger in df[sutun_adi]:
        try:
            float(deger)
        except:
            hatali_degerler.append(deger)
    print(set(hatali_degerler))

In [ ]:
df["TotalCharges"]=df["TotalCharges"].apply(str.strip).replace(" ", np.nan)
df["TotalCharges"]=pd.to_numeric(df["TotalCharges"], downcast="float")

(" ") şeklindeki problemli değerleri tespit ettik.

In [ ]:
df.info()

Ancak onları çıkarınca sütunda 11 eksik değer verdi.

In [ ]:
df.isnull().head(10)

In [ ]:
df.Churn.value_counts()

In [ ]:
df["TotalCharges"].mean()

In [ ]:
df["TotalCharges"].fillna(df["TotalCharges"].mean(), inplace=True)

In [ ]:
df.isnull().sum()*100/df.shape[0]

TotalCharges sütunundaki eksik değerleri tespit edip sütunun ortalaması ile doldurduk. Böylece eksik değer kalmadı.

In [ ]:
df.info()

***Aykırı Değerler***

In [ ]:
plt.rcParams["figure.dpi"]= 100
plt.rcParams["figure.figsize"] = (8,8)
plt.subplot(1,2,1)
plt.boxplot(df["MonthlyCharges"])
plt.title("MonthlyCharges")

plt.subplot(1,2,2)
plt.boxplot(df["TotalCharges"])
plt.title("TotalCharges")
plt.show()

In [ ]:
df.describe(include=["O"])

Box plot üzerinde hemen göremediğimiz aykırı değerler için bir de istatistiksel yöntemler uyguluyoruz.

***Z-score***

In [ ]:

from scipy.stats import zscore

z_scores = zscore(df["TotalCharges"])
for threshold in range(1,5):
    print("Eşik değeri: {}".format(threshold))
    print("Aykırı değerlerin sayısı: {}".format(len((np.where(z_scores > threshold)[0]))))
    print('------')

In [ ]:
z_scores = zscore(df["MonthlyCharges"])
for threshold in range(1,5):
    print("Eşik değeri: {}".format(threshold))
    print("Aykırı değerlerin sayısı: {}".format(len((np.where(z_scores > threshold)[0]))))
    print('------')

***Tukey Metodu***

In [ ]:
q75, q25 = np.percentile(df["TotalCharges"], [75 ,25])
caa = q75 - q25

esik_degerleri = pd.DataFrame()

for esik_degeri in np.arange(1,5,0.5):
    min_deger = q25 - (caa*esik_degeri)
    maks_deger = q75 + (caa*esik_degeri)
    aykiri_deger_sayisi = len((np.where((df["TotalCharges"] > maks_deger)| 
                                       (df["TotalCharges"] < min_deger))[0]))
    esik_degerleri = esik_degerleri.append({'esik_degeri': esik_degeri,'aykiri_deger_sayısı': aykiri_deger_sayisi}, 
                                           ignore_index=True)
esik_degerleri

Çözüm olarak da önce winsorize yöntemini deniyoruz.

In [ ]:
from scipy.stats.mstats import winsorize

winsorize_tcharges = winsorize(df["TotalCharges"], (0, 0.10))

winsorize_tcharges

In [ ]:
plt.boxplot(winsorize_tcharges)
plt.title("TotalCharges'ın kutu grafiği")
plt.show()

Önceki grafikte 8000+ olan üst sınır winsorize işlemi sonunda 6000+ üst sınırına düşmüş.

***Log Dönüşümü***

In [ ]:
plt.figure(figsize = (10,5))

plt.subplot(1,2,1)
plt.hist(df["MonthlyCharges"])
plt.title("MonthlyCharges")

plt.subplot(1,2,2)
plt.hist(np.log(df["MonthlyCharges"]))
plt.title("MonthlyCharges (log dönüşümlü)")
plt.show()

In [ ]:
q75_log, q25_log = np.percentile(np.log(df["TotalCharges"]), [75 ,25])
caa_log = q75_log - q25_log

log_esik_degerleri = pd.DataFrame(columns=['esik_degeri', 'aykiri_deger_sayisi', 'aykiri_deger_sayisi_log'])

for esik_degeri in np.arange(1,5,0.5):
    min_deger_log = q25_log - (caa_log*esik_degeri) 
    maks_deger_log = q75_log + (caa_log*esik_degeri)
    min_deger = q25 - (caa*esik_degeri) #log dönüşümü olmadan
    maks_deger = q75 + (caa*esik_degeri) #log dönüşümü olmadan
    
    aykiri_deger_sayisi_log = len((np.where((np.log(df["TotalCharges"]) > maks_deger_log)| 
                                       (np.log(df["TotalCharges"]) < min_deger_log))[0]))
    aykiri_deger_sayisi = len((np.where((df["TotalCharges"] > maks_deger)| 
                                       (df["TotalCharges"] < min_deger))[0]))
    log_esik_degerleri = log_esik_degerleri.append({'esik_degeri': esik_degeri,
                                                    'aykiri_deger_sayisi' : aykiri_deger_sayisi, #log dönüşümü olmadan
                                                    'aykiri_deger_sayisi_log': aykiri_deger_sayisi_log 
                                                    }, ignore_index=True)
log_esik_degerleri

In [ ]:
log_esik_degerleri.plot(x = 'esik_degeri', y=['aykiri_deger_sayisi_log', 'aykiri_deger_sayisi'])
plt.title('Eşik Değeri & Aykırı Değer Sayısı' )
plt.ylabel('Eşik Değeri')
plt.xlabel('Aykırı Değer Sayısı')
plt.legend(('Logaritmik dönüşümlü', 'Normal'))
plt.show()

**Veri Keşfi**

***Tek Değişkenli Analiz***

In [ ]:
df["winsorize_tenure"] = winsorize(df["tenure"], (0, 0.10))
df["winsorize_MonthlyCharges"] = winsorize(df["MonthlyCharges"], (0, 0.10))
df["winsorize_TotalCharges"] = winsorize(df["TotalCharges"], (0, 0.10))

plt.figure(figsize=(18,6))
degiskenler = ['tenure', 'MonthlyCharges' , 'MonthlyCharges']
winsorize_degiskenler = ['winsorize_tenure', 'winsorize_MonthlyCharges', 'winsorize_TotalCharges']

for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.hist(df[degiskenler[i]])
    plt.title(degiskenler[i])
  

In [ ]:
plt.figure(figsize=(18,6))
for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.hist(df[winsorize_degiskenler[i]])
    plt.title(winsorize_degiskenler[i])
    

In [ ]:
#Kategorik değişkenler

plt.figure(figsize=(14,12), dpi=80)

plt.subplot(3,1,1)
plt.barh(df.groupby("Contract")["Contract"].count().index, 
        df.groupby("Contract")["Contract"].count(),
        color=["blue"])
plt.title("Contract")
plt.subplot(3,1,2)
plt.barh(df.groupby("InternetService")["InternetService"].count().index, 
        df.groupby("InternetService")["InternetService"].count(),
        color=["black"])
plt.title("InternetService")
plt.subplot(3,1,3)
plt.bar(df.groupby("PaymentMethod")["PaymentMethod"].count().index, 
        df.groupby("PaymentMethod")["PaymentMethod"].count(),
       color=["green"])
plt.title("PaymentMethod")
plt.show()

In [ ]:
plt.figure(figsize=(18,4))

plt.subplot(1,3,1)
plt.pie(df.groupby("PaymentMethod")["PaymentMethod"].count(), labels=df.groupby("PaymentMethod")["PaymentMethod"].count().index, autopct='%1.1f%%',
        shadow=True, startangle=90)
plt.title('PaymentMethod')
plt.subplot(1,3,2)
plt.pie(df.groupby("InternetService")["InternetService"].count(), labels=df.groupby("InternetService")["InternetService"].count().index, autopct='%1.1f%%',
        shadow=True, startangle=90)
plt.title('InternetService')
plt.subplot(1,3,3)
plt.pie(df.groupby("Contract")["Contract"].count(), labels=df.groupby("Contract")["Contract"].count().index, autopct='%1.1f%%',
        shadow=True, startangle=90)
plt.title('Contract')
plt.show()

***Çok Değişkenli Analiz***

In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr(),  square=True, annot=True, linewidths=.5, vmin=0, vmax=1, cmap='viridis')

In [ ]:
#Sürekli ve kategorik bir çiftin iki değişkenli analizi
import warnings

warnings.filterwarnings('ignore')

plt.figure(figsize=(18,5))

plt.subplot(1,3,1)
sns.barplot(df["Churn"], df["winsorize_tenure"])
plt.title("tenure")

plt.subplot(1,3,2)
sns.barplot(df["Churn"], df["winsorize_MonthlyCharges"])
plt.title("MonthlyCharges")

plt.subplot(1,3,3)
# draw the heatmap using seaborn.
sns.barplot(df["Churn"], df["winsorize_TotalCharges"])
plt.title("TotalCharge")

plt.show()

Oran olarak baktığımızda Churn sütununa etki bakımından önce tenure, sonra TotalCharges, sonra da MonthlyCharges gelmekte.

In [ ]:
#İki sürekli değişkenin iki değişkenli analizi

plt.figure(figsize=(5,5))
plt.scatter(df["winsorize_MonthlyCharges"], df["winsorize_TotalCharges"])
plt.title("MonthlyCharges & TotalCharges")
plt.xlabel('MonthlyCharges')
plt.ylabel('TotalCharges')
plt.show()

In [ ]:
import scipy.stats as stats
from scipy.stats.mstats import winsorize

churn_status = df["Churn"].unique()
grup_df = df.groupby("Churn")

pd.options.display.float_format = '{:.15f}'.format
for var in ["winsorize_MonthlyCharges", "winsorize_TotalCharges", "winsorize_tenure"]:
    karsilastirma = pd.DataFrame(columns=['grup_1', 'grup_2','istatistik', 'p_degeri'])
    print("{} için karşılaştırma".format(var),end='')
    for i in range(0, len(churn_status)):
        for j in range(i+1, len(churn_status)):
            ttest = stats.ttest_ind(df[df["Churn"]==churn_status[i]][var], 
                                df[df["Churn"]==churn_status[j]][var])
            grup_1 = churn_status[i]
            grup_2 = churn_status[j]
            istatistik = ttest[0]
            p_degeri = ttest[1]
            
            karsilastirma = karsilastirma.append({"grup_1" : grup_1 ,
                                                  "grup_2" : grup_2 ,
                                                  "istatistik": istatistik , 
                                                  "p_degeri" : p_degeri}, ignore_index=True)
    display(karsilastirma)

In [ ]:
#kategorik değişkenler için
plt.figure(figsize=(15,18))
plt.subplot(3,1,1)
sns.countplot(y="PaymentMethod", hue="Churn", data=df)
plt.title("Churn according to PaymentMethod")
plt.ylabel("PaymentMethod")
plt.xlabel("Churn")
plt.subplot(3,1,2)
sns.countplot(y="InternetService", hue="Churn", data=df)
plt.title("Churn according to InternetService")
plt.ylabel("InternetService")
plt.xlabel("Churn")
plt.subplot(3,1,3)
sns.countplot(y="Contract", hue="Churn", data=df)
plt.title("Churn according to Contract")
plt.ylabel("Contract")
plt.xlabel("Churn")
plt.show()

Churn üzerinden karşılaştırdığımızda PaymentMethod için eletronic check'te büyük bir fark gözlenmezken diğer seçenekler de ciddi ayrımlar 
söz konusu. Aynı farktan Contract için de bahsedebiliriz. month-to-month anlaşmalı olanlar arasında one-year ve two-year anlaşmalı olanlara göre daha az bir fark söz konusu.

**Özellik Mühendisliği**

***Mevcut Değişkenleri Dönüştürme ve Yeni Değişkenler Oluşturmak***

In [ ]:
for column in df.columns:
     print("\n" + column)
     print(df[column].value_counts())

Bu özellik sayesinde hangi sütunda hangi değerler var bu değerler kaç kez tekrarlamış görebiliyoruz.

In [ ]:
df["is_male"]= pd.get_dummies(df["gender"], drop_first=True)
df["is_Partner"]= pd.get_dummies(df["Partner"], drop_first=True)
df["is_Dependents"]= pd.get_dummies(df["Dependents"], drop_first=True)
df["is_PhoneService"]= pd.get_dummies(df["PhoneService"], drop_first=True)
df["is_PaperlessBilling"]= pd.get_dummies(df["PaperlessBilling"], drop_first=True)
df["is_Churn"]= pd.get_dummies(df["Churn"], drop_first=True)

2 değere yani yes/no değerlerine sahip olan sütunları daha iyi inceleyebilmek için get_dummies yoluyla 1/0 şeklinde nümerik değerlere dönüştürüyoruz. 

In [ ]:
df.head(2)

In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr())

is_churn değişkeni ile dependents, tenure gibi değişkenler negatif ilişki içinde iken; PaperlessBilling, MonthlyCharges ve SeniorCitizen değişkenleri pozitif bir ilişki içinde.

In [ ]:
#Normal dağılmayan bir değişken ile çalışmak
from scipy.stats import jarque_bera
from scipy.stats import normaltest

pd.options.display.float_format = '{:.5f}'.format

ozellikler = ["MonthlyCharges", "TotalCharges"]
dagilim_testleri = pd.DataFrame(columns=['ozellik', 'jarque_bera_stats', 'jarque_bera_p_value', 
                                         'normal_stats', 'normal_p_value'])

for ozellik in ozellikler:
    jb_stats = jarque_bera(np.log(df[ozellik]))
    norm_stats = normaltest(np.log(df[ozellik]))
    dagilim_testleri = dagilim_testleri.append({"ozellik": ozellik,
                                                "jarque_bera_stats" : jb_stats[0] ,
                                                "jarque_bera_p_value" : jb_stats[1] ,
                                                "normal_stats": norm_stats[0] , 
                                                "normal_p_value" : norm_stats[1]
                                               }, ignore_index=True)
dagilim_testleri

***Normalleştirme ve Standartlaştırma***

In [ ]:
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale

df["norm_winsorize_tenure"] = normalize(np.array(df["winsorize_tenure"]).reshape(1,-1)).reshape(-1,1)
df["norm_winsorize_MonthlyCharges"] = normalize(np.array(df["winsorize_MonthlyCharges"]).reshape(1,-1)).reshape(-1,1)
df["norm_winsorize_TotalCharges"] = normalize(np.array(df["winsorize_TotalCharges"]).reshape(1,-1)).reshape(-1,1)
normal_ozellikler=["winsorize_tenure","norm_winsorize_tenure","winsorize_MonthlyCharges","norm_winsorize_MonthlyCharges",
                    "winsorize_TotalCharges","norm_winsorize_TotalCharges"]


print('Minimum Değer\n',)
print(df[normal_ozellikler].min())
print('\nMaksimum Değer',)
print(df[normal_ozellikler].max())

In [ ]:
plt.figure(figsize=(18,5))

for i in range(3):
    plt.subplot(1,3,i+1)
    plt.scatter(df[normal_ozellikler[2*i]], df[normal_ozellikler[2*i+1]])
    plt.title("Orjinal ve Normalize değerler \n ({})".format(normal_ozellikler[2*i]))
    plt.xlabel("Winsorize Değer")
    plt.ylabel("Normalleştirilmiş winsorize değer")

plt.show()

In [ ]:
df["olcekli_winsorize_tenure"] = scale(df["winsorize_tenure"])
df["olcekli_winsorize_MonthlyCharges"] = scale(df["winsorize_MonthlyCharges"])
df["olcekli_winsorize_TotalCharges"] = scale(df["winsorize_TotalCharges"])
olcekli_ozellikler=["winsorize_tenure","olcekli_winsorize_tenure","winsorize_MonthlyCharges",
                    "olcekli_winsorize_MonthlyCharges","winsorize_TotalCharges","olcekli_winsorize_TotalCharges"]

print('Standart Sapmalar\n',)
print(df[olcekli_ozellikler].std())
print('\nOrtalamalar',)
print(df[olcekli_ozellikler].mean())

In [ ]:
plt.figure(figsize=(18,5))

for i in range(3):
    plt.subplot(1,3,i+1)
    plt.scatter(df[olcekli_ozellikler[2*i]], df[olcekli_ozellikler[2*i+1]])
    plt.title("Orjinal ve Standartlaştırılmış değerler \n ({})".format(olcekli_ozellikler[2*i]))
    plt.xlabel("Winsorize Değer")
    plt.ylabel("Standartlaştırılmış winsorize değer")